## 보스턴 집값 예측

<img src="assets/boston.png">

- 1970년대 매사추세츠주 보스턴에서 우리가 부동산 개발 회사에서 일하고 있다고 기장헤보자. 주거지 프로젝트를 시작하기 전에 평가해 보려고 한다. 우리는 아래와 같이 집의 특성에 따라 가격 견적서를 제공하는 모델을 만들어야 한다.
    - 방의 개수
    - 고용 센터까지의 거리
    - 지역의 경제력 수준
    - 지역 학교의 교사당 학생 수 등

- 우리가 이번 시간을 통해 배우거나 작업할 사항은 아래와 같다. 
    1. 보스턴 집의 가격 데이터를 분석, 연구
    2. 데이터를 훈련용과 테스트용으로 나누기
    3. 다변수 회귀 분석을 수행
    4. 모델의 계수와 잔차 평가
    5. 모델 성능을 개선하기 위해 데이터 변환 사용
    6. 모델을 이용해 부동산 가격을 감정
    
<img src="assets/model.png">

In [1]:
# 필요 모듈 임포트 
import pandas as pd
import numpy as np

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

## 데이터 불러오기

In [4]:
# 데이터 불러오기
data = pd.read_csv('data/boston.csv', index_col=0)
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


## 데이터 탐색

In [5]:
# 데이터 형태 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  PRICE    506 non-null    float64
dtypes: float64(14)
memory usage: 59.3 KB


In [6]:
# 결측치 확인 
data.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64

In [7]:
# 중복값 확인 
data[data.duplicated()]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE


- 1차 데이터 탐색 결과는 아래와 같다 
    - 총 데이터 값 506건 / 컬럼수 14건 
    - 데이터 정보 
        - CRIM(float): 도시별 1인당 범죄율
        - ZN(float): 25,000 평방 피트를 초과하는 부지로 지정된 주거용 토지의 비율
        - INDUS(float): 도시당 비소매 사업 면적 비율
        - CHAS(float): Charles River 더미 변수(= 지역 경계가 강인 경우 1, 그렇지 않은 경우 0)
        - NOX(float): 산화질소 농도(1,000만분의 1)
        - RM(float): 주택당 평균 방 개수
        - AGE(float): 1940년 이전에 건축된 자가 거주 주택의 비율
        - DIS(float): 보스턴 고용 센터 5곳까지의 가중치 거리에 가중치 부여
        - RAD(float): 방사형 고속도로에 대한 접근성 지수
        - TAX(float): `$10,000`당 전체 재산세율
        - PTRATIO(float): 마을별 학생-교사 비율
        - B(float): 1000(Bk - 0.63)^2 여기서 Bk는 마을별 흑인 비율
        - LSTAT(float): 인구의 낮은 지수
        - PRICE(float): 자가 거주 주택의 평균 가치(`$1000`)
    - 데이터 중복값과 NaN 값은 없음 